In [ ]:
import MySQLdb
from pandas.io import sql
import pandas as pd
import numpy as np
conn = MySQLdb.connect(host="localhost", user="root", passwd="Soic@2022Capstone", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
df_concept = pd.read_sql('select * from `concept`', con = conn)
df_concept

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="Soic@2022Capstone", db="mimic4") 
mimiccursor = mimicconn.cursor()

In [ ]:
df_prescriptions = pd.read_sql('select * from prescriptions', con = mimicconn) 
df_prescriptions

In [ ]:
# distint dosage form 
df_dosage_form = pd.read_sql('select distinct form_unit_disp from prescriptions', con = mimicconn) 

# df_dosage_form= df_dosage_form.drop_duplicates()
# len(df_dosage_form)



In [ ]:
df_dosage_form['form_unit_disp'] = df_dosage_form['form_unit_disp'].str.lower()
df_dosage_form = df_dosage_form.drop_duplicates(subset ='form_unit_disp')

In [ ]:
# distinct route
df_route = pd.read_sql('select distinct route from prescriptions', con = mimicconn) 
# df_route = df_route.drop_duplicates()


In [ ]:
df_route['route'] = df_route['route'].str.lower()

In [ ]:
df_route = df_route.drop_duplicates(subset='route')
df_route

In [ ]:
# Getting unique drug names
df_prescriptions_ = pd.read_sql('select distinct drug from prescriptions', con = mimicconn)  

In [ ]:
df_prescriptions_['drug'] = df_prescriptions_['drug'].str.lower()
df_prescriptions_ = df_prescriptions_.drop_duplicates(subset = 'drug')
df_prescriptions_

In [ ]:
df_prescriptions_.loc[df_prescriptions_['drug'] == 'fentanyl citrate']

In [ ]:
df_prescriptions_['datatype_id'] = 4
df_prescriptions_['class_id'] = 3
df_prescriptions_['date_created'] = '2022-01-01'
import uuid
df_prescriptions_['uuid'] = [uuid.uuid4() for x in range(len(df_prescriptions_))]
df_prescriptions_ = df_prescriptions_.rename(columns={'drug':'description'})


In [ ]:
df_prescriptions_ = df_prescriptions_.dropna(subset=['description'])
df_prescriptions_['creator'] = 1

In [ ]:
#Importing unique drug names into concept table

In [ ]:
column_names = ['description', 'datatype_id', 'class_id', 'creator', 'date_created', 'uuid']
df_prescriptions_ = df_prescriptions_.reindex(columns=column_names)

In [ ]:
df_prescriptions_

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_prescriptions_.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_prescriptions_.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

In [ ]:
drug_concept = pd.read_sql('Select * from concept where class_id=3', con = conn)
drug_concept 

In [ ]:
drug_concept['description'].nunique()

# Dealing with dosage form

In [ ]:
df_dosage_form

In [ ]:
df_dosage_form['datatype_id'] = 4
df_dosage_form['class_id'] = 9
df_dosage_form['creator'] = 1
df_dosage_form['date_created'] = '2022-01-01'

import uuid
df_dosage_form['uuid'] = [uuid.uuid4() for x in range(len(df_dosage_form))]
df_dosage_form = df_dosage_form.rename(columns={'form_unit_disp':'description'})


In [ ]:
df_dosage_form = df_dosage_form.dropna(subset=['description'])
# df_dosage_form.to_csv('df_dosage_form.csv')

In [ ]:
df_dosage_form_ = pd.read_csv('df_dosage_form.csv')
df_dosage_form_ = df_dosage_form_[['description_old', 'description']]
df_dosage_form_['description_old'] = df_dosage_form_['description_old'].str.lower()
df_dosage_form_['description'] = df_dosage_form_['description'].str.lower()

In [ ]:
df_dosage_form_

In [ ]:
dosage_dosage= dict(zip(df_dosage_form_.description_old, df_dosage_form_.description))
len(dosage_dosage)

In [ ]:
df_dosage_form['description'] = df_dosage_form['description'].map(dosage_dosage)
df_dosage_form

In [ ]:
df_dosage_form = df_dosage_form.drop_duplicates(subset = 'description')
df_dosage_form

# Dealing with route concepts

In [ ]:
df_route

In [ ]:
df_route['datatype_id'] = 4
df_route['class_id'] = 9
df_route['creator'] = 1
df_route['date_created'] = '2022-01-01'
import uuid
df_route['uuid'] = [uuid.uuid4() for x in range(len(df_route))]
# df_route = df_route.rename(columns={'route':'description'})

In [ ]:
df_route.isnull().sum()

In [ ]:
# df_route = df_route[df_route['description'].notna()]
df_route = df_route.dropna(subset=['route'])
df_route = df_route.drop_duplicates(subset='route')
df_route['route'] = df_route['route'].str.lower()

In [ ]:
df_route

In [ ]:
df_route_ = pd.read_csv('df_route.csv')
df_route_ = df_route_[['description_old', 'description']]
df_route_['description'] = df_route_['description'].str.lower()
df_route_['description_old'] = df_route_['description_old'].str.lower()

In [ ]:
df_route_

In [ ]:
dict_ = {'description_old': ['ad'], 'description': ['ad']}
df = pd.DataFrame(dict_)
df

In [ ]:
df_route_ = pd.concat([df_route_, df], ignore_index = True)
# df_route_ = df_route_.reset_index()

df_route_

In [ ]:
duplicate = df_route_[df_route_.duplicated()]
duplicate

In [ ]:
route_route= dict(zip(df_route_.description_old, df_route_.description))
len(route_route)

In [ ]:
df_route['description'] = df_route['route'].map(route_route)
df_route

In [ ]:
df_route[df_route['description'].isnull()]

In [ ]:
df_route = df_route.drop_duplicates(subset= 'description')
df_route

# Dealing with dose units

In [ ]:
# distint dose units

df_units = pd.read_sql('select distinct dose_unit_rx from prescriptions', con = mimicconn)
df_units['dose_unit_rx'] = df_units['dose_unit_rx'].str.lower()

In [ ]:
df_units

In [ ]:
df_units_rx = pd.read_csv('drug_units.csv')
df_units_rx=df_units_rx[['dose_unit_rx', 'dose_unit']]

In [ ]:
df_units_rx

In [ ]:
df_units_rx.dropna(inplace = True)
df_units_rx

In [ ]:
unique_units = set(df_units_rx['dose_unit'].to_list())
unique_units

In [ ]:
unique_units.remove('-')
unique_units

In [ ]:
list_unique_units = list(unique_units)
len(list_unique_units)

In [ ]:
for index, row in df_units_rx.iterrows():
    if row['dose_unit'] not in list_unique_units:
        df_units_rx=df_units_rx.drop(index)

In [ ]:
df_units_rx['dose_unit_rx'] = df_units_rx['dose_unit_rx'].str.lower()
df_units_rx['dose_unit'] = df_units_rx['dose_unit'].str.lower()
df_units_rx

In [ ]:
dict_unit = dict(zip(df_units_rx.dose_unit_rx, df_units_rx.dose_unit))
len(dict_unit)

In [ ]:
df_units_rx['datatype_id'] = 4
df_units_rx['class_id'] = 9
df_units_rx['creator'] = 1
df_units_rx['date_created'] = '2022-01-01'
import uuid
df_units_rx['uuid'] = [uuid.uuid4() for x in range(len(df_units_rx))]
df_units_rx = df_units_rx.rename(columns={'dose_unit':'description'})
df_units_rx = df_units_rx.drop(columns = ['dose_unit_rx'])


In [ ]:
df_units_rx = df_units_rx.drop_duplicates(subset='description')

In [ ]:
df_units_rx

In [ ]:
concepts_drug_details = pd.concat([df_dosage_form, df_route, df_units_rx],           # Append DataFrames
                      ignore_index = True,
                      sort = False)

In [ ]:
concepts_drug_details = concepts_drug_details.drop(columns=['route'])
concepts_drug_details = concepts_drug_details.drop_duplicates(subset = 'description')
concepts_drug_details

# Importing dosage form, route, and units to concepts

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in concepts_drug_details.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in concepts_drug_details.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

# importing into drug

In [ ]:
df_drug = pd.read_sql('select * from `orders`', con = conn)
df_drug

In [ ]:
drug_details = pd.read_sql('select * from  `concept` where class_id=9', con = conn)
drug_details

In [ ]:
dictdrug = dict(zip(drug_concept.description, drug_concept.concept_id))
len(dictdrug)

In [ ]:
len(dosage_dosage)

In [ ]:
len(route_route)

In [ ]:
len(dict_unit)

In [ ]:
dict_drugdetails = dict(zip(drug_details.description, drug_details.concept_id))
len(dict_drugdetails)

In [ ]:
df_prescriptions_drug = drug_concept[['concept_id', 'description']]
df_prescriptions_drug

In [ ]:
df1 = df_prescriptions[['drug', 'form_unit_disp', 'route', 'dose_unit_rx']]
df1 = df1.rename({'drug':'description'}, axis=1)
df1

In [ ]:
df1['description'] = df1['description'].str.lower()
df1['form_unit_disp'] = df1['form_unit_disp'].str.lower()
df1['route'] = df1['route'].str.lower()
df1['dose_unit_rx'] = df1['dose_unit_rx'].str.lower()
df1

In [ ]:
df1['form_unit_disp'] = df1['form_unit_disp'].map(dosage_dosage)

In [ ]:
df1['route'] = df1['route'].map(route_route)

In [ ]:
df1['dose_unit_rx'] = df1['dose_unit_rx'].map(dict_unit)

In [ ]:
df_drug_merge = pd.merge(df_prescriptions_drug,df1, on= 'description', how='inner')
df_drug_merge

In [ ]:
df_drug_merge = df_drug_merge.drop_duplicates(subset ='description')

In [ ]:
df_drug_merge

In [ ]:
df_drug_merge.isnull().sum()

In [ ]:
df_drug_merge['dosage_form'] = df_drug_merge['form_unit_disp'].map(dict_drugdetails)
df_drug_merge

In [ ]:
df_drug_merge.isnull().sum()

In [ ]:
df_drug_merge['route'] = df_drug_merge['route'].map(dict_drugdetails)
df_drug_merge

In [ ]:
df_drug_merge['dose_limit_units'] = df_drug_merge['dose_unit_rx'].map(dict_drugdetails)
df_drug_merge

In [ ]:
df_drug_merge.isnull().sum()

In [ ]:
df_drug_merge = df_drug_merge.rename(columns={'description':'name'})
df_drug_merge = df_drug_merge.drop(columns = ['form_unit_disp', 'dose_unit_rx'])
df_drug_merge['date_created']= '2022-01-01'
import uuid
df_drug_merge['uuid'] = [uuid.uuid4() for x in range(len(df_drug_merge))]
df_drug_merge['creator'] = 1
df_drug_merge

In [ ]:
df_drug_merge['name'].nunique()

In [ ]:
df_drug_merge.dtypes

In [ ]:
df_drug =  pd.read_sql('select * from `drug`', con = conn)
df_drug

In [ ]:
column_names = ['concept_id', 'name', 'dosage_form', 'route', 'creator', 'date_created', 'uuid', 'dose_limit_units']


df_drug_merge = df_drug_merge.reindex(columns=column_names)
df_drug_merge

In [ ]:
df_drug_merge = df_drug_merge.replace({np.nan: None})
df_drug_merge

In [ ]:
nan_in_col  = df_drug_merge[df_drug_merge['dosage_form'].isna()]
nan_in_col

# Importing into drug table

In [ ]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='Soic@2022Capstone',
                         db='openmrs')

# create cursor
cursor=connection.cursor() 

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_drug_merge.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_drug_merge.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `drug` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        cursor.execute(sql1)
        cursor.execute(sql, tuple(row))
        cursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()